# Построение финансовой модели в Python

где-то здесь в комментариях ссылка на оригинал: https://www.youtube.com/watch?v=ozq2W5iTmx4&list=PLpSi7aN6q3de2mpy5dyOjnG7QUguLt045&index=12&t=2224s&pp=iAQB

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime as dt
from dateutil.relativedelta import relativedelta

import yfinance as yf

from jupyterthemes import jtplot
jtplot.style(theme='onedork', figsize=(15, 9))

In [5]:
def extract_value(cell_range):
    result = []
    for cell in cell_range[0]:
        result.append(abs(cell.value))
    return result


def get_the_average(yearly_values):
    return np.array(yearly_values).mean()


def define_growth(values):
    change_rates = []
    for i in range(1, len(values)):
        change_rate = values[i]/values[i-1] - 1
        change_rates.append(change_rate)
    return change_rates

In [7]:
class Model:
    hist_revenues            = []
    hist_gross_profit        = []
    hist_net_interest        = []
    his_depreciation         = []
    hist_op_profit           = []
    hist_depr                = []
    historical_tax_rate      = []
    hist_NI                  = []
    hist_gross_profit_prct   = []
    hist_net_interest_prct   = []
    hist_depreciation_prct   = []
    hist_op_profit_prct      = []
    hist_depr_prct           = []
    historical_tax_rate_prct = []
    
    normalized= [
        hist_gross_profit_prct,
        hist_net_interest_prct,
        hist_op_profit_prct,
        hist_depr_prct,
    ]

    average_rates = {'gross_profit': 0, 'net_interest': 0, 'operating_profit': 0, 'depreciation': 0,}
    
    def __init__(self, company):
        self.ticker = company

    def define_revenue(self, cell_range):
        self.hist_revenues = extract_value(cell_range)

    def define_gross_profit(self, cell_range):
        self.hist_gross_profit = extract_value(cell_range)

    def define_operating_profit(self, cell_range):
        self.hist_op_profit = extract_value(cell_range)
    
    def calculate_beta(self):
        #get the dates
        today = dt.datetime.today()
        five_years_ago = today - relativedelta(years=5)
        
        #скачиваем данные и выбираем нужные колонки
        stock_price = yf.download(self.ticker, five_years_ago, today, auto_adjust=True)
        stock_price = stock_price['Close']
        index_price = yf.download('SPY', five_years_ago, today, auto_adjust=True)
        index_price = index_price['Close']
        
        #находим изменения в процентах и выбираем значения после первой строчки
        stock_price = stock_price.pct_change(1)[1:]
        index_price = index_price.pct_change(1)[1:]
        x = index_price
        y = stock_price
        #высчитываем бету
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        self.beta = slope